In [2]:
import pandas as pd
import time,datetime
import numpy as np
import glob
from pandas import *
import scipy.optimize
%pylab
%matplotlib inline

#timeseries data
all_data=pd.read_csv('Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})

dic_data = {k: v for k, v in all_data.groupby('date')}

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['unique', 'datetime', 'random', 'test', 'plotting']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
#risk:cov_mat
risk_data=pd.read_csv('cov_mat_2007-01-03.csv')
risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
irows,icols = np.triu_indices(len(risk_sedol),0)
risk_mat[icols,irows]=risk_mat[irows,icols]

In [46]:
import random
import numpy
from operator import attrgetter

mu = 200
#샘플링 수
lambda_ = 1000
#포트폴리오 갯수
ngen = 50
#반복횟수
as_ = dic_data[Timestamp('2007-01-03 00:00:00')]['as'][:10]                                                        # :50 바꿔야함
#알파 스코어
bw = dic_data[Timestamp('2007-01-03 00:00:00')]['bw'][:10]                                                         # :50 바꿔야함
#벤치마크 웨이트
dim = len(as_)
#종목 갯수
rm = risk_mat[:10, :10]                                                                                            # :50 바꿔야함
#상관관계 매트릭스
sector = dic_data[Timestamp('2007-01-03 00:00:00')]['sector'][:10]                                                 # :50 바꿔야함
sec = list(set(dic_data[Timestamp('2007-01-03 00:00:00')]['sector']))
mq = dic_data[Timestamp('2007-01-03 00:00:00')]['mq'][:10]                                                         # :50 바꿔야함
beta = dic_data[Timestamp('2007-01-03 00:00:00')]['beta'][:10]                                                     # :50 바꿔야함
gen = 0
#반복 n번째 수

prob = []
#각 자산들이 포트폴리오 상위 가중치에 있을 확률. len()=492
we_ranks = []
#we_ranks : 각 포트폴리오에서 자산의 가중치의 순위. len(l=ambda_*dim
selected = []
#선택된 포트폴리오. len()=mu
r_r = []
#arz : 생성한 포트폴리오들. len()=lambda_

def generate():
    r_r.clear()
    
    arz = numpy.array([[rand()]*dim]*lambda_)
#   제약식3번,5번, 6번, 7번, 8번 
    i = 0
    while i < lambda_ :                                                                                      #if, for문 말고 제대로된 식으로
        mq_di = [0]*5
        sec_di = [0]*10
        bet = 0
        arz[i] = [max(rand()*0.1 + bw[numpy.where(arz[i] == j)[0][0]]-0.05, 0) for j in arz[i]]
        for j in range(dim) :
            sec_di[sec.index(sector[j])] += arz[i][j] - bw[j]
            mq_di[mq[j]-1] += arz[i][j] - bw[j]
            bet += (arz[i][j] - bw[j])*beta[j]
        if max(sec_di) > 0.1 or min(sec_di) < -0.1 :
            i += -1
        elif max(mq_di) > 0.1 or min(mq_di) < -0.1 :
            i += -1
        elif abs(bet) > 0.1 :
            i += -1
        i += 1

#   생성한 확률을 반영하여 모집단 생성                                                                        #이부분 다 바꿔야함
    if gen >= 1 :
        for i in range(lambda_) :
            for j in range(dim) :
                if rand() > prob[j]/(mu*gen) :
                    arz[i, j] = 0                                                                       #50개보다 작아질경우 어쩌지?
#   확률 모형 0으로 초기화
    elif gen == 0 :
        for i in range(dim) :
            prob.append(0)
#   제약식9번
    for i in range(lambda_) :
        array = numpy.array(arz[i])                                                                 #가중치 제일큰거 60개를 골라?바꿔봐?
        temp = array.argsort()
        arz_ranks = temp.argsort()
        cardi = int(rand()*2+1)                                                                               # *20+50으로 바꿔야함
        for j in range(dim) : 
            if arz_ranks[j] < dim-cardi :
                arz[i][j] = 0

#   제약식 4번
    for i in range(lambda_) :
        arz[i] = [j/sum(arz[i]) for j in arz[i]]
    
#   각 포트폴리오의 risk, as의 합 계산
    for i in range(lambda_) :
        s_rr = numpy.dot(arz[i], rm)
        s_r = numpy.dot(s_rr, arz[i])
        s_a = numpy.dot(arz[i], as_)
        r_r.append(s_r-s_a)
# 각 포트폴리오의 risk-revenue 순위
    array = numpy.array(r_r)
    temp = array.argsort()
    global r_r_ranks
    r_r_ranks = temp.argsort()
    
    return arz



def update(population):
    we_ranks.clear()
    selected.clear()
#   sampling
    for i in range(lambda_) :
        if r_r_ranks[i] < mu :
            selected.append(population[i])
        if r_r_ranks[i] == 0 :
            print('r')
            print(r_r[i])
            
#   각 포트폴리오의 자산 가중치 순위
    array = numpy.array(selected)
    temp = array.argsort()
    we_ranks.append(temp.argsort())
    
#   확률모형 갱신
    for i in range(len(selected)) :
        for j in range(dim) :
            if we_ranks[0][i][j] >= (dim-2) :                                                           #50~70개로 바꿔야함
                prob[j] += 1
    global gen
    gen += 1
    
for i in range(ngen) :
    population = generate()
    update(population)

r
0.033254153648


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in double_scalars


r
0.0345025700903
r
0.0332559535336
r
0.0332847363642


KeyboardInterrupt: 

In [105]:
import random
import numpy
from operator import attrgetter

mu = 200
#샘플링 수
lambda_ = 1000
#포트폴리오 갯수
ngen = 50
#반복횟수
as_ = dic_data[Timestamp('2007-01-03 00:00:00')]['as'][:10]                                                        # :50 바꿔야함
#알파 스코어
bw = dic_data[Timestamp('2007-01-03 00:00:00')]['bw'][:10]                                                         # :50 바꿔야함
#벤치마크 웨이트
dim = len(as_)
#종목 갯수
rm = risk_mat[:10, :10]                                                                                            # :50 바꿔야함
#상관관계 매트릭스
sector = dic_data[Timestamp('2007-01-03 00:00:00')]['sector'][:10]                                                 # :50 바꿔야함
sec = list(set(dic_data[Timestamp('2007-01-03 00:00:00')]['sector']))
mq = dic_data[Timestamp('2007-01-03 00:00:00')]['mq'][:10]                                                         # :50 바꿔야함
beta = dic_data[Timestamp('2007-01-03 00:00:00')]['beta'][:10]                                                     # :50 바꿔야함
gen = 0
#반복 n번째 수

bin_ = []
bin2_ = []

prob = []
#각 자산들이 포트폴리오 상위 가중치에 있을 확률. len()=492
we_ranks = []
#we_ranks : 각 포트폴리오에서 자산의 가중치의 순위. len(l=ambda_*dim
selected = []
#선택된 포트폴리오. len()=mu
r_r = []
#arz : 생성한 포트폴리오들. len()=lambda_

def generate():
    r_r.clear()

    arz = numpy.array([[rand()]*dim]*lambda_)
#   제약식3번,5번, 6번, 7번, 8번 
    i = 0
    while i < lambda_ :                                                                                      #if, for문 말고 제대로된 식으로
        mq_di = [0]*5
        sec_di = [0]*10
        bet = 0
        arz[i] = [max(rand()*0.1 + bw[numpy.where(arz[i] == j)[0][0]]-0.05, 0) for j in arz[i]]
        for j in range(dim) :
            sec_di[sec.index(sector[j])] += arz[i][j] - bw[j]
            mq_di[mq[j]-1] += arz[i][j] - bw[j]
            bet += (arz[i][j] - bw[j])*beta[j]
        if max(sec_di) > 0.1 or min(sec_di) < -0.1 :
            i += -1
        elif max(mq_di) > 0.1 or min(mq_di) < -0.1 :
            i += -1
        elif abs(bet) > 0.1 :
            i += -1
        i += 1

#   생성한 확률을 반영하여 모집단 생성                                                                        #이부분 다 바꿔야함
    if gen >= 1 :
        for i in range(lambda_) :
            for j in range(dim) :
                if rand() > prob[j]/(mu*gen) :
                    arz[i, j] = 0                                                                       #50개보다 작아질경우 어쩌지?
#   확률 모형 0으로 초기화
    elif gen == 0 :
        for i in range(dim) :
            prob.append(0)
#   제약식9번
    for i in range(lambda_) :
        array = numpy.array(arz[i])                                                                 #가중치 제일큰거 60개를 골라?바꿔봐?
        temp = array.argsort()
        arz_ranks = temp.argsort()
        cardi = int(rand()*2+1)                                                                               # *20+50으로 바꿔야함
        for j in range(dim) : 
            if arz_ranks[j] < dim-cardi :
                arz[i][j] = 0

#   제약식 4번
    for i in range(lambda_) :
        arz[i] = [j/sum(arz[i]) for j in arz[i]]
        bin_ = []
        for j in range(dim) :
            if arz[i][j] >0 :
                bin_.append(numpy.where(arz[i] == arz[i][j])[0][0])
        bin2_.append(bin_)
    
    for i in range(lambda_) :
        www = binary_search(bin[i], end_cond=0.001)
        arz[i] = www[0]
    
#   각 포트폴리오의 risk, as의 합 계산
    for i in range(lambda_) :
        s_rr = numpy.dot(arz[i], rm)
        s_r = numpy.dot(s_rr, arz[i])
        s_a = numpy.dot(arz[i], as_)
        r_r.append(s_r-s_a)
#   각 포트폴리오의 risk-revenue 순위
    array = numpy.array(r_r)
    temp = array.argsort()
    global r_r_ranks
    r_r_ranks = temp.argsort()
    
    return arz



def update(population):
    we_ranks.clear()
    selected.clear()
#   sampling
    for i in range(lambda_) :
        if r_r_ranks[i] < mu :
            selected.append(population[i])
        if r_r_ranks[i] == 0 :
            print('r')
            print(r_r[i])
            
#   각 포트폴리오의 자산 가중치 순위
    array = numpy.array(selected)
    temp = array.argsort()
    we_ranks.append(temp.argsort())
    
#   확률모형 갱신
    for i in range(len(selected)) :
        for j in range(dim) :
            if we_ranks[0][i][j] >= (dim-2) :                                                           #50~70개로 바꿔야함
                prob[j] += 1
    global gen
    gen += 1
    
for i in range(ngen) :
    population = generate()
    update(population)

1
[1]
4
[1, 4]
[]
[[1, 4]]
6
[6]
[[1, 4]]
[[1, 4], [6]]
0
[0]
8
[0, 8]
[[1, 4], [6]]
[[1, 4], [6], [0, 8]]
5
[5]
[[1, 4], [6], [0, 8]]
[[1, 4], [6], [0, 8], [5]]
0
[0]
7
[0, 7]
[[1, 4], [6], [0, 8], [5]]
[[1, 4], [6], [0, 8], [5], [0, 7]]
0
[0]
[[1, 4], [6], [0, 8], [5], [0, 7]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0]]
1
[1]
7
[1, 7]
[[1, 4], [6], [0, 8], [5], [0, 7], [0]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7]]
3
[3]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3]]
1
[1]
3
[1, 3]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3]]
0
[0]
6
[0, 6]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6]]
4
[4]
6
[4, 6]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6], [4, 6]]
0
[0]
3
[0, 3]
[[1, 4], [6], [0, 8]

[2, 9]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6], [4, 6], [0, 3], [8], [3, 6], [0, 3], [0, 4], [3], [5], [1], [1, 3], [2, 6], [9], [5], [5], [9], [5, 8], [5, 7], [5], [8], [2], [5, 9], [3], [3], [6], [7, 9], [1, 3], [4], [0], [7], [7], [3, 4], [0], [4], [7], [3], [3, 9], [1], [1], [0, 3], [1, 6], [0, 2], [0, 6], [2, 4], [6, 7], [3, 7], [3], [1], [3], [6, 8], [1], [3, 9], [2], [4], [0, 9], [9]]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6], [4, 6], [0, 3], [8], [3, 6], [0, 3], [0, 4], [3], [5], [1], [1, 3], [2, 6], [9], [5], [5], [9], [5, 8], [5, 7], [5], [8], [2], [5, 9], [3], [3], [6], [7, 9], [1, 3], [4], [0], [7], [7], [3, 4], [0], [4], [7], [3], [3, 9], [1], [1], [0, 3], [1, 6], [0, 2], [0, 6], [2, 4], [6, 7], [3, 7], [3], [1], [3], [6, 8], [1], [3, 9], [2], [4], [0, 9], [9], [2, 9]]
4
[4]
9
[4, 9]
[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6], [4, 6], [0, 3], [8], [3, 6], [0, 3], [0, 4], [3], [5], [1], [1, 3], [

[[1, 4], [6], [0, 8], [5], [0, 7], [0], [1, 7], [3], [1, 3], [0, 6], [4, 6], [0, 3], [8], [3, 6], [0, 3], [0, 4], [3], [5], [1], [1, 3], [2, 6], [9], [5], [5], [9], [5, 8], [5, 7], [5], [8], [2], [5, 9], [3], [3], [6], [7, 9], [1, 3], [4], [0], [7], [7], [3, 4], [0], [4], [7], [3], [3, 9], [1], [1], [0, 3], [1, 6], [0, 2], [0, 6], [2, 4], [6, 7], [3, 7], [3], [1], [3], [6, 8], [1], [3, 9], [2], [4], [0, 9], [9], [2, 9], [4, 9], [4], [1, 4], [7], [3, 6], [5, 7], [0, 3], [1], [8, 9], [2], [4], [2, 9], [0], [4, 8], [2, 5], [7], [1, 2], [3], [4, 7], [7, 9], [5, 6], [4], [5], [3, 4], [3], [1, 9], [3, 7], [1], [6, 7], [2, 8], [0, 5], [1, 2], [0, 1], [7], [1, 6], [5], [5], [7], [4, 5], [0, 3], [2, 9], [6, 8], [8], [5], [7], [5], [4, 6], [4, 5], [3], [3, 4], [3, 9], [7], [6], [0], [0, 5], [3, 8], [0, 1], [0, 8], [8], [1, 5], [4], [3], [1], [3], [1, 8], [2], [0], [7], [2, 7], [5], [3, 6], [2, 3], [2, 8], [3], [7, 8], [7], [1], [2, 8], [5, 7], [4, 6], [7], [2, 6], [2, 3], [1, 6], [6, 8], [1, 8],

KeyboardInterrupt: 

In [101]:
aaa = []
a = [1, 2]
aaa.append(a)
print(aaa)
a = [3, 4]
aaa.append(a)
print(aaa)

[[1, 2]]
[[1, 2], [3, 4]]
